In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
! pip install -U --quiet accelerate
! pip install -U --quiet transformers
! pip install -U --quiet evaluate
! pip install -U --quiet  datasets
! pip install --quiet langchain
! pip install --quiet langchain_community

ERROR: Could not find a version that satisfies the requirement langchain-embeddings (from versions: none)
ERROR: No matching distribution found for langchain-embeddings
ERROR: Could not find a version that satisfies the requirement langchain-text-splitter (from versions: none)
ERROR: No matching distribution found for langchain-text-splitter
ERROR: Could not find a version that satisfies the requirement langchain-vectorstores (from versions: none)
ERROR: No matching distribution found for langchain-vectorstores


In [50]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, DataCollatorWithPadding, Trainer, TrainingArguments,AutoTokenizer
import evaluate
import torch

checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForQuestionAnswering.from_pretrained(checkpoint)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load base model

In [5]:
question = "What is machine learning?"
context = "Machine learning is a field of study in artificial intelligence concerned with the development and study of statistical algorithms."

encoded_input = tokenizer(question, context, return_tensors='pt')

output = model(**encoded_input)

answer_start_index = output.start_logits.argmax()
answer_end_index = output.end_logits.argmax()

predict_answer_tokens = encoded_input.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

''

In [7]:
import nltk
nltk.download('punkt')
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import NLTKTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00


In [41]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="/content/drive/MyDrive/zion.csv")

def preprocess(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=50,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    inputs["start_positions"] = examples["answerStart"]
    inputs["end_positions"] = examples["answerEnd"]
    return inputs

tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)



Map:   0%|          | 0/9 [00:00<?, ? examples/s]

In [51]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,              # total number of training epochs
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    evaluation_strategy="no",
     logging_steps=500,
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    # eval_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
)

trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=20, training_loss=0.0, metrics={'train_runtime': 1.1374, 'train_samples_per_second': 79.125, 'train_steps_per_second': 17.583, 'total_flos': 1148318262000.0, 'train_loss': 0.0, 'epoch': 10.0})

In [66]:
q = "when will humans land on zion ?"
t = tokenizer(q, dataset['train'][8]['context'], return_tensors="pt")

with torch.no_grad():
  outputs = trainer.model(**t.to("cuda"))

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = t.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)


'[CLS] when will humans land on zion? [SEP] the planet zion is the 10th planet of the solar system. it is the planet after pluto and is on the edge of the solar system. the average temperature on zion is - 1000 degrees to - 750 degrees. by 2050, humans are expected to land on'